# Setting up 

In [1]:
import os
import requests
import pandas as pd
import warnings
from dotenv import load_dotenv
from urllib.parse import quote

# Suppress warnings and load environment variables
warnings.filterwarnings('ignore', message='Unverified HTTPS request')
load_dotenv()

# Get Sisense credentials
sisense_url = os.getenv("SISENSE_URL")
username = os.getenv("SISENSE_USER")
password = os.getenv("SISENSE_PASSWORD")

# Create and authenticate session
session = requests.Session()
session.verify = False
session.get(f"{sisense_url}/")
auth_response = session.post(f"{sisense_url}/api/v1/authentication/login", 
                            json={"username": username, "password": password})

if auth_response.status_code == 200:
    # Extract tokens if available
    try:
        resp_data = auth_response.json()
        if 'access_token' in resp_data:
            session.headers.update({
                "Authorization": f"Bearer {resp_data['access_token']}",
                "Content-Type": "application/json",
                "accept": "application/json"
            })
            if 'csrf_token' in resp_data:
                session.headers.update({"X-XSRF-TOKEN": resp_data['csrf_token']})
        print("Authentication successful!")
    except Exception as e:
        print(f"Error setting up headers: {e}")
else:
    print(f"Authentication failed: {auth_response.status_code}")


Authentication successful!


In [3]:
def run_sql_query(query, datasource="RAD_NOVUS_FULL", count=-1):
    """
    Run a SQL query against the specified Sisense datasource
    
    Parameters:
    - query: SQL query string
    - datasource: Name of the Sisense datasource
    - count: Number of rows to return (-1 for all)
    
    Returns:
    - Pandas DataFrame with the results
    """
    # Encode query for URL
    encoded_query = quote(query)
    
    # Build endpoint URL
    endpoint = f"{sisense_url}/api/datasources/{datasource}/sql?count={count}&includeMetadata=true&query={encoded_query}"
    
    # Execute the request
    response = session.get(endpoint)
    
    if response.status_code == 200:
        data = response.json()
        
        if 'values' in data and 'headers' in data:
            # Create DataFrame from the response
            df = pd.DataFrame(data=data["values"], columns=data["headers"])
            print(f"Query executed successfully. Retrieved {len(df)} rows.")
            
            # Display SQL if available in metadata
            if 'metadata' in data and 'sql' in data['metadata']:
                print(f"\nExecuted SQL:\n{data['metadata']['sql']}")
                
            return df
        else:
            print("Unexpected response format")
            print(data)
            return None
    else:
        print(f"Query failed: {response.status_code}")
        print(response.text)
        return None


# C4R_CM_Project_Data_Check_DEV

In [ ]:
""" 
Cube - 'C4R_CM_Project_Data_Check_DEV'. 
Tables: 
- 'attribute_check'; - product_id connected to product features; 
# - 'BM_datacheck'; - project week start and end; 
# - 'num8erz_user'; - table for users login, not important 
- 'pr_chosenfils_dtl'; - Store_id connected to project_id 
- 'pr_chosenupcs'; - project_id connected to UPC_id (probably, this is product_id) 
- 'pr_project'; - project description details; 
- 'Prods_prod'; - product_id and some product details; 
- 'sales_check'; - sales generated by project_id; 
- 'Stocks';  - stores details; 
"""


" \nTables: \n- 'pr_project'; \n- \n"

In [18]:
# Example: Query the stores table
cube = 'C4R_CM_Project_Data_Check_DEV'
table = 'pr_project'
# query = f"SELECT * FROM {table}"
query = f"SELECT data_start as start_date, data_fin as end_date, project_name, project_id FROM {table}"
stores_df = run_sql_query(query, cube)

display(stores_df)


Query executed successfully. Retrieved 6 rows.


,start_date,end_date,project_name,project_id
0,2023-09-10T00:00:00,2024-09-10T00:00:00,Sparkling Demo 09102024,186
1,2023-09-25T00:00:00,2024-09-25T00:00:00,Intermarche_wina,203
2,2023-09-25T00:00:00,2024-09-25T00:00:00,Sparkling Demo_v1,204
3,2023-09-27T00:00:00,2024-09-27T00:00:00,Intermarche_wina2,205
4,2023-10-07T00:00:00,2024-10-07T00:00:00,qwqw,206
5,2023-09-02T00:00:00,2024-09-02T00:00:00,Sparkling_6.1_algo_02.01.2025,207


In [17]:
# Example: Query the stores table
cube = 'C4R_CM_Project_Data_Check_DEV'
table = 'pr_project'
query = f"SELECT data_start as start_date, data_fin as end_date, project_name, project_id FROM {table}"
stores_df = run_sql_query(query, cube)

display(stores_df)


Query executed successfully. Retrieved 6 rows.


,start_date,end_date,project_name,project_id
0,2023-09-10T00:00:00,2024-09-10T00:00:00,Sparkling Demo 09102024,186
1,2023-09-25T00:00:00,2024-09-25T00:00:00,Intermarche_wina,203
2,2023-09-25T00:00:00,2024-09-25T00:00:00,Sparkling Demo_v1,204
3,2023-09-27T00:00:00,2024-09-27T00:00:00,Intermarche_wina2,205
4,2023-10-07T00:00:00,2024-10-07T00:00:00,qwqw,206
5,2023-09-02T00:00:00,2024-09-02T00:00:00,Sparkling_6.1_algo_02.01.2025,207
